<a href="https://colab.research.google.com/github/wyattowalsh/sports-analytics/blob/main/basketball/notebooks/data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align='center'> Basketball Data Collection </h1>

This notebook contains the associated work necessary to collect the data that composes the [***Kaggle Basketball Dataset*** (wyattowalsh/basketball)](https://www.kaggle.com/wyattowalsh/basketball) and serves as the foundation for the [basketball related projects](https://github.com/wyattowalsh/sports-analytics/tree/main/basketball) within my [sports analytics GitHub repository](https://github.com/wyattowalsh/sports-analytics).

One of the goals for the data collection component of this project is to produce a `robust`, *organized* dataset that can grow to as **large of a scale** as possible. You can find an explanation of my solution for storing the files related to the [***Basketball Dataset***](https://www.kaggle.com/wyattowalsh/basketball) below.

<img src="https://unsplash.com/photos/Kv-gAzpUSRg/download?force=true">

## Overview

***Kaggle*** offers many formats of which one can save files to a dataset, which include: `CSV`, `JSON`, `SQLite`, and `Archives`, among others. The platform essentially acts similarly to industrial cloud solutions like *Google Cloud Platform's* (**GCP**) ***Cloud Storage*** or *Amazon Web Service's* (**AWS**) ***S3*** albeit with a **100GB** storage capacity. ***Kaggle*** datasets as well as these industrial solutions can be considered as broad object/file storage and in certain data engineering paradigms can serve as data lakes. 

It seems that many state-of-the-art (SOTA) data storage solutions pivot around an organizational-wide data lake (of which itself allows for general object storage) that has multiple inputs (*"tributaries"*) both streaming into and routinely added to the overall lake. One benefit of this paradigm is that the lake facilitates the storage of both structured (tabular) and unstructured (image, video, audio, text, etc) data. This can prove useful because, as time progresses, new techniques for extracting useful information from unstructured data can be utilized. Thus it also seems like a good idea to hold onto all extracted data, if possible. 

***Kaggle*** datasets can serve as data lakes through the archival process or simply by storing data files in their raw file format. This certainly serves as a strong foundation for building a &#8212; one day in the future &#8212; <b><i>"big data"</i></b> collection. 

However, there is further work that can be done in configuring ***Kaggle*** datasets to enable additional platform functionality as well as improved storage efficiency. Structured data, whether structured upon extraction or structured through some pre-processing, can be stored in a ***SQLite*** database (`.sqlite` file type) as opposed to storing individual files such as `CSVs` or `JSONs` within the dataset. Thus, a single database file is stored as an object within the dataset, enabling additional functionality. One easily discerned advantage with storing in ***SQLite*** is that histograms of the distribution of across continuous variables are given directly within ***Kaggle***. 

As this project moves forward, I hope to collect a large collection of both structured and unstructured data. I hope that the ***SQLite*** database (`basketball.sqlite`) can serve to house the structured data in an efficient, useful format, similarly to the [***European Soccer Database***](https://www.kaggle.com/hugomathien/soccer).

## View System Information

In [ ]:
print("********************** CUDA Version ********************** \n - \n")
!nvcc --version
print("********************** CPU Info ********************** \n - \n")
!cat /proc/cpuinfo
print("********************** CPU Count ********************** \n - \n")
import os
print(os.cpu_count())
print("********************** GPU Info ********************** \n - \n")
!nvidia-smi
print("********************** Python Version ********************** \n - \n")
!python -V

********************** CUDA Version ********************** 
 - 

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0
********************** CPU Info ********************** 
 - 

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0
siblings	: 4
core id		: 0
cpu cores	: 2
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_singl

## Prepare Development Environment

### Clone Project Repository and Install Dependencies

In [ ]:
# remove sample data and clone repo
!rm -r sample_data/
!rm -r sports-analytics/
!git clone https://github.com/wyattowalsh/sports-analytics.git

# change directory to directory that contains this notebook
%cd /content/sports-analytics/basketball/notebooks/

# install dependencies
!pip install -r ../../dependencies/basketball/data_collection.txt

rm: cannot remove 'sample_data/': No such file or directory
rm: cannot remove 'sports-analytics/': No such file or directory
Cloning into 'sports-analytics'...
remote: Enumerating objects: 387, done.
remote: Counting objects: 100% (387/387), done.
remote: Compressing objects: 100% (278/278), done.
remote: Total 387 (delta 144), reused 268 (delta 64), pack-reused 0
Receiving objects: 100% (387/387), 768.27 KiB | 8.00 MiB/s, done.
Resolving deltas: 100% (144/144), done.
/content/sports-analytics/basketball/notebooks


### Import Dependencies and Enable Tools

In [1]:
# nba_api dependencies
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import commonplayerinfo, playercareerstats, teamdetails, leaguegamefinder

# datascience stack
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import sqlite3 as sql

# system utility stack
import os
import time
import urllib
from functools import partial

pd.options.display.max_columns = None

# # Upload kaggle.json to /content/
from google.colab import files 
uploaded = files.upload()

# Move and change permissions as needed, allowing for import
!mkdir -p ~/.kaggle/ && cp kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
import kaggle

# # change directory to directory that contains this notebook
%cd /content/sports-analytics/basketball/notebooks/

# # utilize Colab Monitor
# from urllib.request import urlopen
# exec(urlopen("http://colab-monitor.smankusors.com/track.py").read())
# _colabMonitor = ColabMonitor().start()

/content/sports-analytics/basketball/notebooks


## Collect Data

### Connect to Database

In [2]:
conn = sql.connect('../data/basketball.sqlite')

### Players

#### Get Players DataFrame and Type ID as String

In [ ]:
df_players = pd.DataFrame(players.get_players()).astype({'id': 'str'})
df_players

,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False
...,...,...,...,...,...
4496,1627790,Ante Zizic,Ante,Zizic,True
4497,78647,Jim Zoet,Jim,Zoet,False
4498,78648,Bill Zopf,Bill,Zopf,False
4499,1627826,Ivica Zubac,Ivica,Zubac,True


In [ ]:
df_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4501 entries, 0 to 4500
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          4501 non-null   object
 1   full_name   4501 non-null   object
 2   first_name  4501 non-null   object
 3   last_name   4501 non-null   object
 4   is_active   4501 non-null   bool  
dtypes: bool(1), object(4)
memory usage: 145.2+ KB


#### Add Dataframe as Table to Database, Unless it Already Exists

In [ ]:
try:
    df_players.to_sql('Player', conn, index=False)
except:
    pass

### Teams

#### Get Teams DataFrame, Type ID as String and Convert Year to Datetime

In [ ]:
df_teams = pd.DataFrame(teams.get_teams()).astype({'id': 'str'})
df_teams['year_founded'] =  pd.to_datetime(df_teams['year_founded'], format='%Y').dt.year # convert year to datetime type
df_teams.head()

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Atlanta,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966


In [ ]:
df_teams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            30 non-null     object
 1   full_name     30 non-null     object
 2   abbreviation  30 non-null     object
 3   nickname      30 non-null     object
 4   city          30 non-null     object
 5   state         30 non-null     object
 6   year_founded  30 non-null     int64 
dtypes: int64(1), object(6)
memory usage: 1.8+ KB


#### Add Dataframe as Table to Database, Unless it Already Exists

In [ ]:
try:
    df_teams.to_sql('Team', conn, index=False)
except:
    pass

### Common Player Information

In [ ]:
# define function to extract common player info for a single player
def get_common_player_info(player_id, proxies):
    # define helpful variables
    no_res = True
    proxy_collection_counter = 0
    proxy_index = 0
    # while no response
    while no_res:
        # try getting a response without a proxy
        try:
            res = commonplayerinfo.CommonPlayerInfo(player_id=player_id, timeout=3).get_data_frames()
            no_res = False
            print(player_id)
            break
        except:
            # if that fails
            while no_res:
                # try getting with a certain proxy
                try: 
                    res = commonplayerinfo.CommonPlayerInfo(player_id=player_id, proxy="http://" + proxies[proxy_index], timeout=3).get_data_frames()
                    no_res = False
                    break
                except:
                    # if that fails, move on to next proxy unless out of proxies
                    if (proxy_index + 1) >= len(proxies):
                        # unless tried proxies 5 times
                        if proxy_collection_counter < 6:
                            # if out of proxies: get more proxies, fix counters, and try without a proxy again
                            proxy_index = 0
                            proxy_collection_counter = proxy_collection_counter + 1
                            print(player_id, ' failed {} times'.format(proxy_collection_counter))
                            proxies = [str(proxy).split('\\')[0][2:] for proxy in urllib.request.urlopen("https://api.proxyscrape.com/v2/?request=getproxies&protocol=http&timeout=1000&country=all&ssl=yes&anonymity=all&simplified=true").readlines()]
                            break
                        else:
                            return None
                    else:
                        proxy_index = proxy_index + 1
                        
    # merge the common player info and player headline stats and drop timeframe                   
    res_df = pd.merge(res[0], res[1], how='left', left_on=['PERSON_ID', 'DISPLAY_FIRST_LAST'], right_on=['PLAYER_ID', 'PLAYER_NAME'])
    res_df = res_df.drop(['TimeFrame'], axis=1)
    return res_df

# get proxies
proxies = [str(proxy).split('\\')[0][2:] for proxy in urllib.request.urlopen("https://api.proxyscrape.com/v2/?request=getproxies&protocol=http&timeout=1000&country=all&ssl=yes&anonymity=all&simplified=true").readlines()]

# get common player info for each player in the db
dfs = []
player_ids = pd.read_sql('SELECT id FROM Player', conn)['id'].values
dfs = [get_common_player_info(player_id=player_id, proxies=proxies) for player_id in player_ids]
df = pd.concat(dfs)
df.head()

76001
76002
76003
51
1505
949
76005
76006
76007
203518
101165
76008
76009
76010
203112
76011
76012
200801
1629121
203919
149
203500
912
1628389
1629061
76015
202399
201167
200772
76016
201336
76017
201582
76018
203006
1629152
202374
76019
76020
203128
202332
200746
76021
1626146
724
2042
76022
201570
1629734
2349
1629638
76024
1628959
76028
1628960
1628386
706
1628443
202730
76027
2124
76025
951
2754
76029
200984
76030
201165
308
1747
1824
680
732
202329
200811
76034
2365
2431
101187
202079
76035
76036
1507
76037
944
246
202341
76040
1626147
72
76041
203937
76042
76043
98
76045
76046
201583
1000
335
76048
101149
76049
1628387
76050
1512
203507
1628961
203648
2546
21
201202
203544
1628384
203951
2737
76053
76054
2425
1627853
76055
2240
2772
76056
76057
769
76061
2220
353
200756
76060
76059
76062
76063
76064
2306
201589
76065
1628503
201600
355
173
76068
76069
1088
76070
76071
278
201571
200788
1134
76073
76074
138
76076
1895
76078
202970
201965
1629028
101235
203569
202337
76079
76080
1

76786
1051
203197
76788
76789
76790
76791
1619
1727
76793
201188
2200
76794
423
930
200752
76797
1845
1627771
1002
202087
761
101153
1904
76800
202331
76801
1627875
76802
76803
76804
76805
76806
76807
200789
76808
1626780
76809
76810
201959
76811
201592
201821
1628385
1628983
2109
383
76812
76813
288
76815
600014
76817
76818
1938
1921
76819
202148
76820
76821
76822
76823
76824
76825
1901
2601
76826
203497
76827
76828
76829
924
101155
76830
76831
2400
76832
1864
203462
1629164
76833
203932
2732
204079
201569
76834
76835
101162
76836
202726
76837
76838
76839
1628984
37
101121
76843
76841
76842
101211
1626203
76844
97
101122
76847
258
76850
202
285
265
270
203924
1626170
76852
1514
76853
1629055
76854
76855
201189
1070
1516
1026
76859
1627982
76860
76861
76862
76863
81
76865
920
201980
101209
203110
203475
101123
203210
1629750
201145
76867
76868
76869
600011
1036
76872
76873
76874
76875
76877
76876
76878
201192
76879
2584
201590
101158
76880
76881
76882
2696
76883
76884
76885
76886
76887

203114
77589
2036
77590
600012
77591
77593
77594
101223
101139
2032
77596
1527
2545
77598
1889
292
77605
77603
1802
203121
77602
77599
77600
77601
1626259
2034
932
203113
397
77604
168
77606
201988
371
202407
200794
1629003
114
77609
77610
65
77612
202703
77613
1954
1628378
77615
77616
417
77614
203183
203502
1628513
77618
77619
1749
211
77621
77622
1737
2040
77623
1629690
77624
77625
77626
77627
2752
1628370
600006
202328
77628
1626242
77630
376
200081
77631
1629760
77632
1628500
202734
77634
77633
77635
1630
77636
77637
929
1629630
203961
77639
77640
1629752
77641
77642
77644
356
202721
77647
1628537
202693
77646
1628420
201043
2242
202694
200747
77648
77649
201627
132
1972
77652
77653
77654
734
202700
1628405
2069
203102
297
202389
77657
1626144
77658
203498
77659
77660
201957
904
77662
942
204098
77664
77665
785
49
77668
77669
77672
203513
77670
77671
203122
77673
77674
2211
1627749
2436
1627750
77675
441
145
77676
203315
1629058
203488
77677
1054
87
982
939
1629004
1626122  failed

201987
202685
78395
1627756
78396
78397
78398
78399
78401
78400
89
1496
78406
1627832
78402
78403
1629020
78405
78404
78407
2128
2237
2760
202363
202349
78409
710
1521
1626173
78411
919
78413
78414
202686
78415
78416
101111
1629216
78418
78417
200790
1844
78419
78420
78421
78422
203943
2063
78423
2582
1037
78425
2761
202696
2756
1629731
2548
78426
101144
78427
2402
78428
78429
1629021
203079
78430
78431
78432
78433
952
78434
78435
78437
78436
201611
78438
78439
202689
78440
78441
955
78442
1629022
202322
1112
2222
961
78443
739
1627820
78444
78445
78446
101190
777
78447
78448
78450
78449
2575
1628476
202954
78453
78454
369
78455
78456
203810
78457
78458
78459
78460
1627866
78461
203933
202378
78462
101124
78463
1627395
78469
200827
78464
78465
1540
78466
78467
1629023
78468
78470
78471
1629139
1629682
201208
201228
2248
320
1628778
78474
78473
203146
204033
204037
221
78475
1629683
201602
78477
78478
892
1627821
185
1627362
78479
1067
101110
78481
78482
201603
78483
78484
78485
78486
1

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,PLAYER_SLUG,BIRTHDATE,SCHOOL,COUNTRY,LAST_AFFILIATION,HEIGHT,WEIGHT,SEASON_EXP,JERSEY,POSITION,ROSTERSTATUS,GAMES_PLAYED_CURRENT_SEASON_FLAG,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CODE,TEAM_CITY,PLAYERCODE,FROM_YEAR,TO_YEAR,DLEAGUE_FLAG,NBA_FLAG,GAMES_PLAYED_FLAG,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,PLAYER_ID,PLAYER_NAME,PTS,AST,REB,ALL_STAR_APPEARANCES,PIE
0,76001,Alaa,Abdelnaby,Alaa Abdelnaby,"Abdelnaby, Alaa",A. Abdelnaby,alaa-abdelnaby,1968-06-24T00:00:00,Duke,USA,Duke/USA,6-10,240,4,30,Forward,Inactive,N,1610612757,Trail Blazers,POR,blazers,Portland,HISTADD_alaa_abdelnaby,1990,1994,N,Y,Y,1990,1,25,76001,Alaa Abdelnaby,5.7,0.3,3.3,0,NaN
0,76002,Zaid,Abdul-Aziz,Zaid Abdul-Aziz,"Abdul-Aziz, Zaid",Z. Abdul-Aziz,zaid-abdul-aziz,1946-04-07T00:00:00,Iowa State,USA,Iowa State/USA,6-9,235,9,54,Center,Inactive,N,1610612745,Rockets,HOU,rockets,Houston,HISTADD_zaid_abdul-aziz,1968,1977,N,Y,Y,1968,1,5,76002,Zaid Abdul-Aziz,9.0,1.2,8.0,0,NaN
0,76003,Kareem,Abdul-Jabbar,Kareem Abdul-Jabbar,"Abdul-Jabbar, Kareem",K. Abdul-Jabbar,kareem-abdul-jabbar,1947-04-16T00:00:00,UCLA,USA,UCLA/USA,7-2,225,19,33,Center,Inactive,N,1610612747,Lakers,LAL,lakers,Los Angeles,HISTADD_kareem_abdul-jabbar,1969,1988,N,Y,Y,1969,1,1,76003,Kareem Abdul-Jabbar,24.6,3.6,11.2,18,NaN
0,51,Mahmoud,Abdul-Rauf,Mahmoud Abdul-Rauf,"Abdul-Rauf, Mahmoud",M. Abdul-Rauf,mahmoud-abdul-rauf,1969-03-09T00:00:00,Louisiana State,USA,Louisiana State/USA,6-1,162,8,1,Guard,Inactive,N,1610612743,Nuggets,DEN,nuggets,Denver,mahmoud_abdul-rauf,1990,2000,N,Y,Y,1990,1,3,51,Mahmoud Abdul-Rauf,14.6,3.5,1.9,0,NaN
0,1505,Tariq,Abdul-Wahad,Tariq Abdul-Wahad,"Abdul-Wahad, Tariq",T. Abdul-Wahad,tariq-abdul-wahad,1974-11-03T00:00:00,San Jose State,France,San Jose State/France,6-6,235,6,9,Forward-Guard,Inactive,N,1610612758,Kings,SAC,kings,Sacramento,tariq_abdul-wahad,1997,2003,N,Y,Y,1997,1,11,1505,Tariq Abdul-Wahad,7.8,1.1,3.3,0,NaN


In [ ]:
dff = df.copy()
df.rename(columns = {'PERSON_ID':'ID'}, inplace = True)
df['ID'] = df["ID"].astype(str)
df['BIRTHDATE'] = df['BIRTHDATE'].astype(str)
df['HEIGHT'] = df['HEIGHT'].apply(lambda x: int(x.split('-')[0])*12 + int(x.split('-')[1]) \
                                  if len(x.strip()) > 0 else np.nan)
df['WEIGHT'] = df['WEIGHT'].apply(lambda x: int(x) if len(x.strip()) > 0 else np.nan)
df['TEAM_ID'] = df['TEAM_ID'].astype(str)
df['FROM_YEAR'] = df['FROM_YEAR'].astype(str)
df['TO_YEAR'] = df['TO_YEAR'].astype(str)
df['DRAFT_YEAR'] = df['DRAFT_YEAR'].astype(str)
df['DRAFT_ROUND'] = df['DRAFT_ROUND'].astype(str)
df['DRAFT_NUMBER'] = df['DRAFT_NUMBER'].astype(str)
df = df.drop(['PLAYER_ID', 'PLAYER_NAME'], axis=1)
df['PTS'] = pd.to_numeric(df['PTS'], errors='coerce')
df['AST'] = pd.to_numeric(df['AST'], errors='coerce')
df['REB'] = pd.to_numeric(df['REB'], errors='coerce')
df['ALL_STAR_APPEARANCES'] = pd.to_numeric(df['ALL_STAR_APPEARANCES'], errors='coerce')
df['PIE'] = pd.to_numeric(df['PIE'], errors='coerce')
df = df.reset_index(drop=True)

In [ ]:
df.describe()

,HEIGHT,WEIGHT,SEASON_EXP,PTS,AST,REB,ALL_STAR_APPEARANCES,PIE
count,4403.000000,4399.000000,4500.000000,4485.000000,4485.000000,4193.000000,4056.000000,429.000000
mean,78.070634,211.146852,4.295556,6.357101,1.436299,2.981588,0.343195,0.084002
std,3.638698,27.075019,4.492292,4.925010,1.409078,2.288022,1.474006,0.088903
min,65.000000,133.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.500000
25%,75.000000,190.000000,1.000000,2.800000,0.500000,1.400000,0.000000,0.067000
50%,78.000000,210.000000,3.000000,5.100000,1.000000,2.400000,0.000000,0.089000
75%,81.000000,230.000000,7.000000,8.600000,1.900000,4.000000,0.000000,0.112000
max,91.000000,360.000000,22.000000,31.800000,11.200000,22.900000,18.000000,0.211000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 37 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ID                                4500 non-null   object 
 1   FIRST_NAME                        4500 non-null   object 
 2   LAST_NAME                         4500 non-null   object 
 3   DISPLAY_FIRST_LAST                4500 non-null   object 
 4   DISPLAY_LAST_COMMA_FIRST          4500 non-null   object 
 5   DISPLAY_FI_LAST                   4500 non-null   object 
 6   PLAYER_SLUG                       4500 non-null   object 
 7   BIRTHDATE                         4500 non-null   object 
 8   SCHOOL                            4497 non-null   object 
 9   COUNTRY                           4500 non-null   object 
 10  LAST_AFFILIATION                  4500 non-null   object 
 11  HEIGHT                            4403 non-null   float64
 12  WEIGHT

In [ ]:
df.to_sql('Player_Attributes', conn, index=False, if_exists='replace')

In [ ]:
try:
    df.to_sql('Player_Attributes', conn, index=False)
except:
    pass

#### Upload to Kaggle

In [ ]:
!kaggle datasets version -p ../data -m "adding common player info"

Starting upload for file basketball.sqlite
100%|███████████████████████████████████████| 1.70M/1.70M [00:07<00:00, 225kB/s]
Upload successful: basketball.sqlite (2MB)
Dataset version creation error: You have exceeded the max category limit


### Team Details

In [ ]:
# define function to extract common player info for a single player
def get_team_details(team_id, proxies):
    # define helpful variables
    no_res = True
    proxy_collection_counter = 0
    proxy_index = 0
    # while no response
    while no_res:
        # try getting a response without a proxy
        try:
            res = teamdetails.TeamDetails(team_id, timeout=3).get_data_frames()
            no_res = False
            print(team_id)
            break
        except:
            # if that fails
            while no_res:
                # try getting with a certain proxy
                try: 
                    res = teamdetails.TeamDetails(team_id, proxy="http://" + proxies[proxy_index], timeout=3).get_data_frames()
                    no_res = False
                    break
                except:
                    # if that fails, move on to next proxy unless out of proxies
                    if (proxy_index + 1) >= len(proxies):
                        # unless tried proxies 5 times
                        if proxy_collection_counter < 5:
                            # if out of proxies: get more proxies, fix counters, and try without a proxy again
                            proxy_index = 0
                            proxy_collection_counter = proxy_collection_counter + 1
                            print(team_id, ' failed {} times'.format(proxy_collection_counter))
                            proxies = [str(proxy).split('\\')[0][2:] for proxy in urllib.request.urlopen("https://api.proxyscrape.com/v2/?request=getproxies&protocol=http&timeout=1000&country=all&ssl=yes&anonymity=all&simplified=true").readlines()]
                            break
                        else:
                            return None
                    else:
                        proxy_index = proxy_index + 1
                        
    # merge the common player info and player headline stats and drop timeframe  
    dfs = res
    df = dfs[0]
    try:
        df['FACEBOOK_WEBSITE_LINK'] = dfs[2].loc[dfs[2]['ACCOUNTTYPE'] == 'Facebook']['WEBSITE_LINK'].values[0]
    except:
        df['FACEBOOK_WEBSITE_LINK'] = np.nan
    try:
        df['INSTAGRAM_WEBSITE_LINK'] = dfs[2].loc[dfs[2]['ACCOUNTTYPE'] == 'Instagram']['WEBSITE_LINK'].values[0]
    except:
        df['INSTAGRAM_WEBSITE_LINK'] = np.nan
    try:
        df['TWITTER_WEBSITE_LINK'] = dfs[2].loc[dfs[2]['ACCOUNTTYPE'] == 'Twitter']['WEBSITE_LINK'].values[0]
    except:
        df['TWITTER_WEBSITE_LINK'] = np.nan
    df.rename(columns = {'TEAM_ID':'ID'}, inplace = True)
    df['ID'] = df["ID"].astype(str)
    df['YEARFOUNDED'] = df['YEARFOUNDED'].astype(str)
    df['ARENACAPACITY'] = pd.to_numeric(df['ARENACAPACITY'], errors='coerce')
    df_1 = dfs[1]
    df_1.rename(columns = {'TEAM_ID':'ID'}, inplace = True)
    df_1['ID'] = df_1["ID"].astype(str)
    df_1['YEARFOUNDED'] = df_1['YEARFOUNDED'].astype(str)
    df_1['YEARACTIVETILL'] = df_1['YEARACTIVETILL'].astype(str)
    return [df, df_1]

# get proxies
proxies = [str(proxy).split('\\')[0][2:] for proxy in urllib.request.urlopen("https://api.proxyscrape.com/v2/?request=getproxies&protocol=http&timeout=1000&country=all&ssl=yes&anonymity=all&simplified=true").readlines()]

# get common player info for each player in the db
dfs = []
team_ids = pd.read_sql('SELECT id FROM Team', conn)['id'].values
dfs = [get_team_details(team_id, proxies=proxies) for team_id in team_ids]
df_0 = pd.concat([df[0] for df in dfs])
df_1 = pd.concat([df[1] for df in dfs])
display(df_0.head())
df_1.head()

1610612737
1610612738
1610612739
1610612740
1610612741
1610612742
1610612743
1610612744
1610612745
1610612746
1610612747
1610612748
1610612749
1610612750
1610612751
1610612752
1610612753
1610612754
1610612755
1610612756
1610612757
1610612758
1610612759
1610612760
1610612761
1610612762
1610612763
1610612764
1610612765
1610612766


,ID,ABBREVIATION,NICKNAME,YEARFOUNDED,CITY,ARENA,ARENACAPACITY,OWNER,GENERALMANAGER,HEADCOACH,DLEAGUEAFFILIATION,FACEBOOK_WEBSITE_LINK,INSTAGRAM_WEBSITE_LINK,TWITTER_WEBSITE_LINK
0,1610612737,ATL,Hawks,1949,Atlanta,State Farm Arena,18729.0,Tony Ressler,Travis Schlenk,Nate McMillan,Erie Bayhawks,https://www.facebook.com/hawks,https://instagram.com/atlhawks,https://twitter.com/ATLHawks
0,1610612738,BOS,Celtics,1946,Boston,TD Garden,18624.0,Wyc Grousbeck,Danny Ainge,Brad Stevens,Maine Red Claws,https://www.facebook.com/bostonceltics,https://instagram.com/celtics,https://twitter.com/celtics
0,1610612739,CLE,Cavaliers,1970,Cleveland,Rocket Mortgage FieldHouse,20562.0,Dan Gilbert,Koby Altman,JB Bickerstaff,Canton Charge,https://www.facebook.com/Cavs,https://instagram.com/cavs,https://twitter.com/cavs
0,1610612740,NOP,Pelicans,2002,New Orleans,Smoothie King Center,NaN,Tom Benson,Trajan Langdon,Stan Van Gundy,No Affiliate,https://www.facebook.com/PelicansNBA,https://instagram.com/pelicansnba,https://twitter.com/PelicansNBA
0,1610612741,CHI,Bulls,1966,Chicago,United Center,21711.0,Jerry Reinsdorf,Arturas Karnisovas,Billy Donovan,Windy City Bulls,https://www.facebook.com/chicagobulls,https://instagram.com/chicagobulls,https://twitter.com/chicagobulls


,ID,CITY,NICKNAME,YEARFOUNDED,YEARACTIVETILL
0,1610612737,Atlanta,Hawks,1968,2019
1,1610612737,St. Louis,Hawks,1955,1967
2,1610612737,Milwaukee,Hawks,1951,1954
3,1610612737,Tri-Cities,Blackhawks,1949,1950
0,1610612738,Boston,Celtics,1946,2019


In [ ]:
df_0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 0 to 0
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      30 non-null     object 
 1   ABBREVIATION            30 non-null     object 
 2   NICKNAME                30 non-null     object 
 3   YEARFOUNDED             30 non-null     object 
 4   CITY                    30 non-null     object 
 5   ARENA                   30 non-null     object 
 6   ARENACAPACITY           20 non-null     float64
 7   OWNER                   30 non-null     object 
 8   GENERALMANAGER          30 non-null     object 
 9   HEADCOACH               30 non-null     object 
 10  DLEAGUEAFFILIATION      30 non-null     object 
 11  FACEBOOK_WEBSITE_LINK   30 non-null     object 
 12  INSTAGRAM_WEBSITE_LINK  30 non-null     object 
 13  TWITTER_WEBSITE_LINK    30 non-null     object 
dtypes: float64(1), object(13)
memory usage: 3.5+ 

In [ ]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 0 to 2
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              60 non-null     object
 1   CITY            60 non-null     object
 2   NICKNAME        60 non-null     object
 3   YEARFOUNDED     60 non-null     object
 4   YEARACTIVETILL  60 non-null     object
dtypes: object(5)
memory usage: 2.8+ KB


In [ ]:
try:
    df_0.to_sql('Team_Attributes', conn, index=False)
except:
    pass
try:
    df_1.to_sql('Team_History', conn, index=False)
except:
    pass

#### Upload to Kaggle

In [ ]:
!kaggle datasets version -p ../data -m "adding team details"

Starting upload for file basketball.sqlite
100%|███████████████████████████████████████| 1.70M/1.70M [00:04<00:00, 435kB/s]
Upload successful: basketball.sqlite (2MB)
Dataset version creation error: You have exceeded the max category limit


In [ ]:
# define function to extract common player info for a single player
def get_league_games(season_id, proxies):
    # define helpful variables
    no_res = True
    proxy_collection_counter = 0
    proxy_index = 0
    # while no response
    while no_res:
        # try getting a response without a proxy
        try:
            res = leaguegamefinder.LeagueGameFinder(season_nullable=season_id, timeout=7).get_data_frames()
            no_res = False
            print(season_id)
            break
        except:
            # if that fails
            while no_res:
                # try getting with a certain proxy
                try: 
                    res = leaguegamefinder.LeagueGameFinder(season_nullable=season_id, proxy="http://" + proxies[proxy_index], timeout=7).get_data_frames()
                    no_res = False
                    break
                except:
                    # if that fails, move on to next proxy unless out of proxies
                    if (proxy_index + 1) >= len(proxies):
                        # unless tried proxies 5 times
                        if proxy_collection_counter < 5:
                            # if out of proxies: get more proxies, fix counters, and try without a proxy again
                            proxy_index = 0
                            proxy_collection_counter = proxy_collection_counter + 1
                            print(season_id, ' failed {} times'.format(proxy_collection_counter))
                            proxies = [str(proxy).split('\\')[0][2:] for proxy in urllib.request.urlopen("https://api.proxyscrape.com/v2/?request=getproxies&protocol=http&timeout=2500&country=all&ssl=yes&anonymity=all&simplified=true").readlines()]
                            break
                        else:
                            return None
                    else:
                        proxy_index = proxy_index + 1
                        
    # merge the common player info and player headline stats and drop timeframe      
    df = res[0]
    game_ids = df["GAME_ID"].unique()         

    def get_df(df, game_id):
      season_id = df['SEASON_ID'].values[0]
      rows = df.loc[df["GAME_ID"] == game_id].drop('GAME_ID', "SEASON_ID", axis=1)
      row_0 = rows.iloc[[0]]
      row_1 = rows.iloc[[1]]

      def rename_cols(df):
        if "vs" in df['MATCHUP'].values[0]:
          df.columns = [x + '_HOME' for x in df.columns]
        else:
          df.columns = [x + '_AWAY' for x in df.columns]
        return df
        row_0 = rename_cols(row_0)
        row_1 = rename_cols(row_1)
        df = pd.concat([df_0, df_1], axis=1)
        cols = list(df.columns.values)
        cols = ['GAME_ID', "SEASON_ID"] + cols
        df["GAME_ID"] = game_id
        df["SEASON_ID"] = season_id
        df = df[cols]
        return df
      df = pd.concat([get_df(df, game_id) for game_id in game_ids])
      return df

# get proxies
proxies = [str(proxy).split('\\')[0][2:] for proxy in urllib.request.urlopen("https://api.proxyscrape.com/v2/?request=getproxies&protocol=http&timeout=1000&country=all&ssl=yes&anonymity=all&simplified=true").readlines()]

# get common player info for each player in the db
dfs = []
player_ids = pd.read_sql('SELECT id FROM Player', conn)['id'].values
season_ids = [str(x) + "-" + str(x+1)[2:4] for x in range(1946, 2021)]
dfs = [get_league_games(season_id=season_id , proxies=proxies) for season_id in season_ids[0:2]]
df = pd.concat(dfs)
df

In [46]:
dfs = pd.concat([get_df(df, game_id) for game_id in game_ids])

KeyError: ignored

In [11]:
dfs[0][0]

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,41986,1610612747,LAL,Los Angeles Lakers,0048600071,1987-06-14,LAL vs. BOS,W,240,106,45,93,0.484,0,5,0.000,16,23,0.696,13,31,44,33,11,9,11,28,None
1,41986,1610612738,BOS,Boston Celtics,0048600071,1987-06-14,BOS @ LAL,L,240,93,33,81,0.407,0,5,0.000,27,31,0.871,13,29,42,21,6,7,18,25,None
2,41986,1610612738,BOS,Boston Celtics,0048600070,1987-06-11,BOS vs. LAL,W,240,123,46,89,0.517,6,10,0.600,25,31,0.806,13,33,46,33,6,3,10,21,None
3,41986,1610612747,LAL,Los Angeles Lakers,0048600070,1987-06-11,LAL @ BOS,L,240,108,43,95,0.453,4,8,0.500,18,22,0.818,17,23,40,16,7,4,12,26,None
4,41986,1610612738,BOS,Boston Celtics,0048600069,1987-06-09,BOS vs. LAL,L,240,106,45,86,0.523,3,11,0.273,13,16,0.813,9,28,37,26,5,8,12,21,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025,21986,1610612755,PHL,Philadelphia 76ers,0028600002,1986-10-31,PHL vs. IND,W,240,108,43,84,0.512,2,3,0.667,20,30,0.667,18,30,48,26,7,5,25,27,None
2026,21986,1610612744,GOS,Golden State Warriors,0028600009,1986-10-31,GOS @ PHX,L,240,123,49,99,0.495,1,6,0.167,24,31,0.774,18,27,45,26,15,3,18,31,None
2027,21986,1610612764,WAS,Washington Bullets,0028600003,1986-10-31,WAS @ BOS,L,240,102,37,85,0.435,0,1,0.000,28,36,0.778,20,18,38,18,10,6,22,25,None
2028,21986,1610612759,SAN,San Antonio Spurs,0028600007,1986-10-31,SAN @ DEN,L,240,104,43,93,0.462,2,6,0.333,16,24,0.667,16,18,34,27,7,4,17,22,None


In [10]:
dfs[0][0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2030 entries, 0 to 2029
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   SEASON_ID          2030 non-null   object 
 1   TEAM_ID            2030 non-null   int64  
 2   TEAM_ABBREVIATION  2030 non-null   object 
 3   TEAM_NAME          2030 non-null   object 
 4   GAME_ID            2030 non-null   object 
 5   GAME_DATE          2030 non-null   object 
 6   MATCHUP            2030 non-null   object 
 7   WL                 2030 non-null   object 
 8   MIN                2030 non-null   int64  
 9   PTS                2030 non-null   int64  
 10  FGM                2030 non-null   int64  
 11  FGA                2030 non-null   int64  
 12  FG_PCT             2030 non-null   float64
 13  FG3M               2030 non-null   int64  
 14  FG3A               2030 non-null   int64  
 15  FG3_PCT            1957 non-null   float64
 16  FTM                2030 

In [4]:
[str(x) + "-" + str(x+1)[2:4] for x in range(1946, 2021)]

['1946-47',
 '1947-48',
 '1948-49',
 '1949-50',
 '1950-51',
 '1951-52',
 '1952-53',
 '1953-54',
 '1954-55',
 '1955-56',
 '1956-57',
 '1957-58',
 '1958-59',
 '1959-60',
 '1960-61',
 '1961-62',
 '1962-63',
 '1963-64',
 '1964-65',
 '1965-66',
 '1966-67',
 '1967-68',
 '1968-69',
 '1969-70',
 '1970-71',
 '1971-72',
 '1972-73',
 '1973-74',
 '1974-75',
 '1975-76',
 '1976-77',
 '1977-78',
 '1978-79',
 '1979-80',
 '1980-81',
 '1981-82',
 '1982-83',
 '1983-84',
 '1984-85',
 '1985-86',
 '1986-87',
 '1987-88',
 '1988-89',
 '1989-90',
 '1990-91',
 '1991-92',
 '1992-93',
 '1993-94',
 '1994-95',
 '1995-96',
 '1996-97',
 '1997-98',
 '1998-99',
 '1999-00',
 '2000-01',
 '2001-02',
 '2002-03',
 '2003-04',
 '2004-05',
 '2005-06',
 '2006-07',
 '2007-08',
 '2008-09',
 '2009-10',
 '2010-11',
 '2011-12',
 '2012-13',
 '2013-14',
 '2014-15',
 '2015-16',
 '2016-17',
 '2017-18',
 '2018-19',
 '2019-20',
 '2020-21']

In [47]:
pd.concat(dfs, ignore_index=True)

,GAME_ID,TEAM_ID_HOME,TEAM_ABBREVIATION_HOME,TEAM_NAME_HOME,GAME_DATE_HOME,MATCHUP_HOME,WL_HOME,MIN_HOME,PTS_HOME,FGM_HOME,FGA_HOME,FG_PCT_HOME,FG3M_HOME,FG3A_HOME,FG3_PCT_HOME,FTM_HOME,FTA_HOME,FT_PCT_HOME,OREB_HOME,DREB_HOME,REB_HOME,AST_HOME,STL_HOME,BLK_HOME,TOV_HOME,PF_HOME,PLUS_MINUS_HOME,TEAM_ID_AWAY,TEAM_ABBREVIATION_AWAY,TEAM_NAME_AWAY,GAME_DATE_AWAY,MATCHUP_AWAY,WL_AWAY,MIN_AWAY,PTS_AWAY,FGM_AWAY,FGA_AWAY,FG_PCT_AWAY,FG3M_AWAY,FG3A_AWAY,FG3_PCT_AWAY,FTM_AWAY,FTA_AWAY,FT_PCT_AWAY,OREB_AWAY,DREB_AWAY,REB_AWAY,AST_AWAY,STL_AWAY,BLK_AWAY,TOV_AWAY,PF_AWAY,PLUS_MINUS_AWAY
0,0048600071,1.610613e+09,LAL,Los Angeles Lakers,1987-06-14,LAL vs. BOS,W,240.0,106.0,45.0,93.0,0.484,0.0,5.0,0.000,16.0,23.0,0.696,13.0,31.0,44.0,33.0,11.0,9.0,11.0,28.0,None,1.610613e+09,BOS,Boston Celtics,1987-06-14,BOS @ LAL,L,240.0,93.0,33.0,81.0,0.407,0.0,5.0,0.0,27.0,31.0,0.871,13.0,29.0,42.0,21.0,6.0,7.0,18.0,25.0,None
1,0048600070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.610613e+09,LAL,Los Angeles Lakers,1987-06-11,LAL @ BOS,L,240.0,108.0,43.0,95.0,0.453,4.0,8.0,0.5,18.0,22.0,0.818,17.0,23.0,40.0,16.0,7.0,4.0,12.0,26.0,None
2,0048600070,1.610613e+09,BOS,Boston Celtics,1987-06-11,BOS vs. LAL,W,240.0,123.0,46.0,89.0,0.517,6.0,10.0,0.600,25.0,31.0,0.806,13.0,33.0,46.0,33.0,6.0,3.0,10.0,21.0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0048600069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.610613e+09,LAL,Los Angeles Lakers,1987-06-09,LAL @ BOS,W,240.0,107.0,41.0,85.0,0.482,2.0,4.0,0.5,23.0,32.0,0.719,15.0,31.0,46.0,17.0,4.0,4.0,11.0,16.0,None
4,0048600069,1.610613e+09,BOS,Boston Celtics,1987-06-09,BOS vs. LAL,L,240.0,106.0,45.0,86.0,0.523,3.0,11.0,0.273,13.0,16.0,0.813,9.0,28.0,37.0,26.0,5.0,8.0,12.0,21.0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024,0028600005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.610613e+09,LAC,Los Angeles Clippers,1986-10-31,LAC @ SAC,L,240.0,106.0,39.0,90.0,0.433,0.0,3.0,0.0,28.0,35.0,0.800,12.0,19.0,31.0,18.0,11.0,3.0,12.0,27.0,None
2025,0028600002,1.610613e+09,PHL,Philadelphia 76ers,1986-10-31,PHL vs. IND,W,240.0,108.0,43.0,84.0,0.512,2.0,3.0,0.667,20.0,30.0,0.667,18.0,30.0,48.0,26.0,7.0,5.0,25.0,27.0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2026,0028600002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.610613e+09,IND,Indiana Pacers,1986-10-31,IND @ PHL,L,240.0,104.0,39.0,83.0,0.470,0.0,3.0,0.0,26.0,35.0,0.743,14.0,21.0,35.0,26.0,14.0,4.0,14.0,26.0,None
2027,0028600003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.610613e+09,WAS,Washington Bullets,1986-10-31,WAS @ BOS,L,240.0,102.0,37.0,85.0,0.435,0.0,1.0,0.0,28.0,36.0,0.778,20.0,18.0,38.0,18.0,10.0,6.0,22.0,25.0,None


In [31]:
pd.concat(a, axis=1)

,TEAM_ID_HOME,TEAM_ABBREVIATION_HOME,TEAM_NAME_HOME,GAME_DATE_HOME,MATCHUP_HOME,WL_HOME,MIN_HOME,PTS_HOME,FGM_HOME,FGA_HOME,FG_PCT_HOME,FG3M_HOME,FG3A_HOME,FG3_PCT_HOME,FTM_HOME,FTA_HOME,FT_PCT_HOME,OREB_HOME,DREB_HOME,REB_HOME,AST_HOME,STL_HOME,BLK_HOME,TOV_HOME,PF_HOME,PLUS_MINUS_HOME,TEAM_ID_AWAY,TEAM_ABBREVIATION_AWAY,TEAM_NAME_AWAY,GAME_DATE_AWAY,MATCHUP_AWAY,WL_AWAY,MIN_AWAY,PTS_AWAY,FGM_AWAY,FGA_AWAY,FG_PCT_AWAY,FG3M_AWAY,FG3A_AWAY,FG3_PCT_AWAY,FTM_AWAY,FTA_AWAY,FT_PCT_AWAY,OREB_AWAY,DREB_AWAY,REB_AWAY,AST_AWAY,STL_AWAY,BLK_AWAY,TOV_AWAY,PF_AWAY,PLUS_MINUS_AWAY
0,1.610613e+09,LAL,Los Angeles Lakers,1987-06-14,LAL vs. BOS,W,240.0,106.0,45.0,93.0,0.484,0.0,5.0,0.0,16.0,23.0,0.696,13.0,31.0,44.0,33.0,11.0,9.0,11.0,28.0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.610613e+09,BOS,Boston Celtics,1987-06-14,BOS @ LAL,L,240.0,93.0,33.0,81.0,0.407,0.0,5.0,0.0,27.0,31.0,0.871,13.0,29.0,42.0,21.0,6.0,7.0,18.0,25.0,None
